### Code for training Convolutional Neural Network

First, load the required libraries.

In [1]:
import tensorflow as tf
import pandas as pd
import xarray as xr
import glob as glob
import numpy as np
import sys
import pickle

from datetime import timedelta, datetime
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Reshape, Add, ReLU, Conv2DTranspose, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Model
from tensorflow.keras.regularizers import l2
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

/home/rjackson/.conda/envs/pydda_env/lib/python3.9/site-packages/xarray/backends/cfgrib_.py:27: UserWarning: Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. Try `import cfgrib` to get the full error message
  warnings.warn(


Next, we load the EPA AirNow data into a Pandas dataframe.

In [2]:
air_now_data = glob.glob('/lcrc/group/earthscience/rjackson/epa_air_now/*.csv')
air_now_df = pd.concat(map(pd.read_csv, air_now_data))
air_now_df['datetime'] = pd.to_datetime(air_now_df['DateObserved'] + ' 00:00:00')
air_now_df = air_now_df.set_index('datetime')
air_now_df = air_now_df.sort_index()
print(air_now_df['CategoryNumber'].values.min())

# How many timesteps ahead should we predict?
lag = 0
site = "hou"

# Insert the path to your cropped dataset here
input_merra_path = '/lcrc/group/earthscience/rjackson/MERRA2/hou_reduced/'

1


This next block of code will grab the EPA AirNow classification given a particular datetime.

In [3]:
def get_air_now_label(time):
    if np.min(np.abs((air_now_df.index - time))) > timedelta(days=1):
        return np.nan
    ind = np.argmin(np.abs(air_now_df.index - time))
    return air_now_df['CategoryNumber'].values[ind]

This next block of code loads the cropped MERRA2 dataset. 

load_data takes in a list of aerosol species as inputs. The aerosol species are the abbreviations used in the MERRA2 dataset (i.e. BC, SO4, SO2, DU, SS, OC, etc.). In addition, the *test_data_size* parameter specifies the fraction of the dataset to be set aside as a testing dataset for training.

The returned parameters from load_data are:

   * x_dataset_train, x_dataset_test: Dictionaries containing the test and training dataset inputs
   * y_train, y_test: The training and test labels corresponding to x_dataset
   * shape: The shape of the input x data.
    

In [4]:
def load_data(species, test_data_size=0.20):
    ds = xr.open_mfdataset(input_merra_path + '%sCMASS*.nc' % species).sortby('time')
    print(ds)
    times = np.array(list(map(pd.to_datetime, ds.time.values)))
    x = ds["%sCMASS" % species].values
    old_shape = x.shape
    scaler = StandardScaler()
    scaler.fit(np.reshape(x, (old_shape[0], old_shape[1] * old_shape[2])))
    x = scaler.transform(
            np.reshape(x, (old_shape[0], old_shape[1] * old_shape[2])))
    x = np.reshape(x, old_shape)
    inputs = np.zeros((old_shape[0], old_shape[1], old_shape[2], 3))
    inputs[:, :, :, 0] = x
    ds.close()
    if species == "SO4" or species == "DMS" or species == "SO2":
       inp = "SU"
    else:
       inp = species
    ds = xr.open_mfdataset(input_merra_path + '%sFLUXU*.nc' % inp).sortby('time')
    x2 = ds["%sFLUXU" % inp].values
    scaler = StandardScaler()
    scaler.fit(np.reshape(x2, (old_shape[0], old_shape[1] * old_shape[2])))
    x2 = scaler.transform(
            np.reshape(x2, (old_shape[0], old_shape[1] * old_shape[2])))
    x2 = np.reshape(x2, old_shape)
    inputs[:, :, :, 1] = x2
    ds.close()
    ds = xr.open_mfdataset(input_merra_path + '%sFLUXV*.nc' % inp).sortby('time')
    x2 = ds["%sFLUXV" % inp].values
    scaler = StandardScaler()
    scaler.fit(np.reshape(x2, (old_shape[0], old_shape[1] * old_shape[2])))
    x2 = scaler.transform(
            np.reshape(x2, (old_shape[0], old_shape[1] * old_shape[2])))
    x2 = np.reshape(x2, old_shape)
    inputs[:, :, :, 2] = x2
    classification = np.array(list(map(get_air_now_label, times)))
    where_valid = np.isfinite(classification)
    inputs = inputs[where_valid, :, :, :]
    classification = classification[where_valid]
    # Use classification from x days ahead
    if lag > 0:
        classification = classification[lag*8:]
        inputs = inputs[:lag*8, :, : :]

    y = tf.one_hot(classification, 5).numpy()
    x_train, x_test, y_train, y_test = train_test_split(
            inputs, y, test_size=test_data_size)
    shape = inputs.shape
    x_dataset_train = {'input_1': np.squeeze(x_train[:, :, :, 0]),
            'input_2': np.squeeze(x_train[:, :, :, 1]),
            'input_3': np.squeeze(x_train[:, :, :, 2])}
    x_dataset_test = {'input_1': np.squeeze(x_test[:, :, :, 0]),
            'input_2': np.squeeze(x_test[:, :, :, 1]),
            'input_3': np.squeeze(x_test[:, :, :, 2])}

    return x_dataset_train, x_dataset_test, y_train, y_test, shape

Construct the parallel CNN model. This procedure takes in 3 parameters:

   * shape: the shape of the input data as a tuple
   * the_dict: A dictionary containing the model hyperparameters
   * dataset: The input dataset returned by *load_data*

In [5]:
def classifier_model(shape, the_dict, dataset):
    width = shape[2]
    height = shape[1]
    mpool_1s = []
    in_layers = []
    dict_keys = list(dataset.keys())
    for j in range(len(dict_keys)):
        inp_layer = Input(shape=(height, width, 1), name=dict_keys[j])
        mpool_1 = inp_layer
        in_layers.append(inp_layer)
        for i in range(the_dict['num_layers']):
            conv2d_1 = Conv2D(the_dict['num_channels'],
                (2, 2), activation=the_dict['activation'], padding='same',
                kernel_initializer='he_normal')(mpool_1)
            conv2d_1 = BatchNormalization()(conv2d_1)
            mpool_1 = MaxPooling2D((2, 2))(conv2d_1)
        mpool_1s.append(Flatten()(mpool_1))
    flat_1 = Add()(mpool_1s)
    
    for i in range(the_dict['num_dense_layers']):
        flat_1 = Dense(the_dict['num_dense_nodes'], activation='relu'
                )(flat_1)
        flat_1 = BatchNormalization()(flat_1)

    output = Dense(5, activation="softmax", name="class")(flat_1)
    
    return Model(in_layers, output)

Do the model training and data loading.

In [6]:
def run(config: dict):
    x_ds_train = {}
    x_ds_test = {}
    y_train = []
    y_test = []
    # Adjust to the species you want to have
    species_list = config['species_list']
    
    for species in species_list:
        print(species)
        x_ds_train1, x_ds_test1, y_train, y_test, shape = load_data(species)
        x_ds_train.update(x_ds_train1)
        x_ds_test.update(x_ds_test1)
    
    model = classifier_model(shape, config, x_ds_test)
    model.compile(optimizer=Adam(lr=config["learning_rate"]),
        loss="categorical_crossentropy", metrics=['acc'])
    model.summary()

    # AQI classes inbalanced, need weights
    class_weight = {0: 1, 1: 2, 2: 20, 3: 90, 4: 510}
    history = model.fit(
            x_ds_train, y_train, 
            validation_data=(x_ds_test, y_test), epochs=config["num_epochs"],
            batch_size=config["batch_size"], class_weight=class_weight)
    model.save('classifier-%s-lag-%d' % (site, lag))
    return history.history

Call the model training code 

In [ ]:
default_config = {
        "num_epochs": 10,        # Number of training epochs. Recommended to use 200+ fo, 10 here is to make cookbook short
        "num_channels": 128,     # Number of channels in Conv2D layer 
        "learning_rate": 0.0001, # Learning rate for training
        "num_dense_nodes": 8,    # Number of nodes in each dense layer of the classifier
        "num_dense_layers": 3,   # Number of layers in classifier portion
        "activation": "relu",    # Activation function used (ReLU is typically good)
        "batch_size": 10,        # Batch size for training
        "num_layers": 2,         # Number of Conv2D layers in feature extractor
        # Aerosol species to use in model training
        "species_list": ['SS', 'SO4', 'SO2', 'OC', 'DU', 'DMS', 'BC']}

history = run(default_config)

SS


/home/rjackson/.conda/envs/pydda_env/lib/python3.9/site-packages/xarray/backends/plugins.py:61: RuntimeWarning: Engine 'cfgrib' loading failed:
Cannot find the ecCodes library
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


<xarray.Dataset>
Dimensions:    (lon: 32, lat: 20, time: 90744)
Coordinates:
  * lon        (lon) float64 -105.0 -104.4 -103.8 ... -86.88 -86.25 -85.62
  * lat        (lat) float64 25.0 25.5 26.0 26.5 27.0 ... 33.0 33.5 34.0 34.5
  * time       (time) datetime64[ns] 2010-01-01T00:30:00 ... 2020-05-08T23:30:00
Data variables:
    SSCMASS    (time, lat, lon) float32 dask.array<chunksize=(8760, 20, 32), meta=np.ndarray>
    SSCMASS25  (time, lat, lon) float32 dask.array<chunksize=(8760, 20, 32), meta=np.ndarray>


2022-10-21 13:52:27.969958: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-21 13:52:28.469454: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38396 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:07:00.0, compute capability: 8.0


SO4
<xarray.Dataset>
Dimensions:   (lon: 32, lat: 20, time: 90744)
Coordinates:
  * lon       (lon) float64 -105.0 -104.4 -103.8 -103.1 ... -86.88 -86.25 -85.62
  * lat       (lat) float64 25.0 25.5 26.0 26.5 27.0 ... 33.0 33.5 34.0 34.5
  * time      (time) datetime64[ns] 2010-01-01T00:30:00 ... 2020-05-08T23:30:00
Data variables:
    SO4CMASS  (time, lat, lon) float32 dask.array<chunksize=(8760, 20, 32), meta=np.ndarray>
SO2
<xarray.Dataset>
Dimensions:   (lon: 32, lat: 20, time: 90744)
Coordinates:
  * lon       (lon) float64 -105.0 -104.4 -103.8 -103.1 ... -86.88 -86.25 -85.62
  * lat       (lat) float64 25.0 25.5 26.0 26.5 27.0 ... 33.0 33.5 34.0 34.5
  * time      (time) datetime64[ns] 2010-01-01T00:30:00 ... 2020-05-08T23:30:00
Data variables:
    SO2CMASS  (time, lat, lon) float32 dask.array<chunksize=(8760, 20, 32), meta=np.ndarray>
OC
<xarray.Dataset>
Dimensions:  (lon: 32, lat: 20, time: 90744)
Coordinates:
  * lon      (lon) float64 -105.0 -104.4 -103.8 -103.1 ... -86.88 -8

/home/rjackson/.conda/envs/pydda_env/lib/python3.9/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 20, 32, 1)]  0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 20, 32, 1)]  0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 20, 32, 1)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 20, 32, 128)  640         ['input_1[0][0]']                
                                                                                              

2022-10-21 13:56:35.550070: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8201
2022-10-21 13:56:39.240906: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-10-21 13:56:43.187227: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


5171/5171 [==============================] - 37s 5ms/step - loss: 17.7415 - acc: 0.1739 - val_loss: 1.6914 - val_acc: 0.2393
Epoch 2/10
5171/5171 [==============================] - 23s 5ms/step - loss: 13.6297 - acc: 0.2408 - val_loss: 1.7013 - val_acc: 0.2383
Epoch 3/10
5171/5171 [==============================] - 23s 4ms/step - loss: 11.5479 - acc: 0.2662 - val_loss: 1.5164 - val_acc: 0.2916
Epoch 4/10
5171/5171 [==============================] - 23s 5ms/step - loss: 10.2620 - acc: 0.2709 - val_loss: 1.5523 - val_acc: 0.2610
Epoch 5/10
5171/5171 [==============================] - 23s 5ms/step - loss: 9.2578 - acc: 0.2775 - val_loss: 1.5187 - val_acc: 0.2725
Epoch 6/10
5171/5171 [==============================] - 23s 4ms/step - loss: 8.3513 - acc: 0.2832 - val_loss: 1.6070 - val_acc: 0.2544
Epoch 7/10
5171/5171 [==============================] - 23s 4ms/step - loss: 7.5896 - acc: 0.2909 - val_loss: 1.4485 - val_acc: 0.2831
Epoch 8/10
5171/5171 [==============================] - 23s 4m